In [1]:
import time

import vk
import os
from pyvis.network import Network
from dotenv import load_dotenv

In [ ]:
load_dotenv("./config/secrets.env")

token = ''
try:
    token = os.environ["VK_ACCESS_TOKEN"]
except KeyError:
    print("You need to enter your access token from https://vkhost.github.io/ into env VK_ACCESS_TOKEN")
    exit(1)

vk_api = vk.API(token, v='5.131')

In [4]:
max_friends_per_friend = 100
def get_user_graph(user_id):
    friends = vk_api.friends.get(user_id=user_id)['items']
    friends_of_friends_dict = {}
    friends_set = set()
    friends_edges_set = set()

    for friend in friends:
        try:
            friends_of_friends = vk_api.friends.get(user_id=friend)['items']
        except Exception:
            continue # если пользователь удален
        if len(friends_of_friends) > max_friends_per_friend:
            continue
        friends_of_friends_dict[friend] = friends_of_friends
        
        friends_set.add(friend)
        for friend_of_friend in friends_of_friends_dict[friend]:
            friends_set.add(friend_of_friend)
            friend_pair = [friend, friend_of_friend]
            friend_pair.sort()
            friends_edges_set.add(tuple(friend_pair))
        time.sleep(0.1) # потому что ограничение на количество запросов в секунду

    return friends_set, friends_edges_set

In [5]:
id_anton = 243317609
id_matvey = 207227130

anton_friend_nodes, anton_friend_edges = get_user_graph(id_anton)
matvey_friend_nodes, matvey_friend_edges = get_user_graph(id_matvey)

In [6]:
friend_set = anton_friend_nodes.union(matvey_friend_nodes)
friend_edge_set = anton_friend_edges.union(matvey_friend_edges)

In [7]:
graph = Network(notebook=True)

for friend in friend_set:
    graph.add_node(friend, label=str(friend))

for friend_pair in friend_edge_set:
    graph.add_edge(friend_pair[0], friend_pair[1])

In [8]:
graph.show("graph.html")

graph.html
